In [0]:
import re
import collections
import nltk
from nltk.corpus import stopwords
import numpy as np
import math
import pickle,gzip
import scipy

In [0]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
f = open("/content/drive/My Drive/CS25025/HW5/wiki-text.txt","r")
corpus = f.read()

In [0]:
# make a list of words from Wikipedia corpus

In [0]:
wiki = re.findall(r'\w+', corpus.lower())

In [0]:
# remove words that appear less than 500 times 

In [0]:
counter = collections.Counter(wiki)
counter = collections.Counter(dict(filter(lambda x: x[1] >= 8, counter.items())))

In [0]:
# remove all the words that appear in the stopwords list of nltk package

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
counter = collections.Counter(dict(filter(lambda x: x[0] not in stop_words, counter.items())))
list(counter.items())[:5]

[('anarchism', 157),
 ('originated', 73),
 ('term', 777),
 ('abuse', 46),
 ('first', 3444)]

In [0]:
counter["physics"]

173

In [0]:
# Applying the filter of "at least 10 appearances", we get the vocabulary whose 
# length is close to 15000

In [0]:
print(len(list(counter.keys())))

15424


In [0]:
V = [i for i in list(counter.keys())]

In [0]:
print("physics" in V)

True


In [0]:
# Part (a)

In [0]:
# use symmetric window size of 5 to find all word-context pairs 

In [0]:
win = 5
sp = collections.Counter()
for i in range(len(wiki)):
    section = wiki[max(0,i-win):i]+wiki[i+1:min(len(wiki),i+1+win)]
    for context in section:
        sp[(wiki[i], context)] += 1

In [0]:
# find the number of word-context pairs

In [0]:
c = sum(list(sp.values()))
print(c)

19671140


In [0]:
# Create a function that calculates N^p(w)

In [0]:
def compute_np(sp, w):
    return sum([sp[x] for x in sp if w in x])

In [0]:
# In this part, we need to find M_ij as given in the instructions. However, even 
# with GPU, the computation time is beyond reasonable. Therefore, I decided to 
# find alternative ways to calculate N^p(w). 
# The intuition is that the more often a word appears in the corpus, the more 
# often it will appear in S_p. I believe that the correlation could be somewhat 
# linear, and hence we can normalize this value. This way, we can find N^p(w) 
# value much easier. For example, 

In [0]:
print(compute_np(sp, "abuse") / counter["abuse"])
print(compute_np(sp, "first") / counter["first"])

20.0
19.98199767711963


In [0]:
# Indeed, the intuition was right and the N^p(w) value is 20 times the counter[w]
# value. From now one, let's use 20*counter[w] to find N^p(w), instead of 
# repeating the compute_np(sp, w) function so many times. 

In [0]:
def find_M(w_i, w_j, sp):
    return (math.log(((sp[(w_i,w_j)]+1)*c)/((counter[w_i]*20)*(counter[w_j]*20))))

In [0]:
# create a function that finds the PMI matrix 

In [0]:
def compute_PMI(sp, V):
    M = np.zeros((len(V),len(V)))
    for i in range(len(V)):
        for j in range(len(V)):
            M[i,j] = find_M(V[i], V[j], sp)   
    return M

In [0]:
M = compute_PMI(sp, V)

In [0]:
M[:5, :5]

array([[ 4.76824443,  2.14964053,  0.70095035,  1.9183114 , -2.39743607],
       [ 2.14964053,  2.22227972,  1.80320896,  3.37724494, -0.53303742],
       [ 0.70095035,  1.80320896,  0.32553124,  1.41772914, -0.56264341],
       [ 1.9183114 ,  3.37724494,  1.41772914,  3.1459158 , -0.47668448],
       [-2.39743607, -0.53303742, -0.56264341, -0.47668448, -1.34244441]])

In [0]:
# Part (b)

In [0]:
# factorize the matrix M to obtain word embeddings. Take the k-SVD of M with 
# k = 50 using scipy package

In [0]:
(U,s,V) = scipy.sparse.linalg.svds(scipy.sparse.csr_matrix(M), k = 50)

In [0]:
U[:5, :5]

array([[-0.02253238,  0.0134218 ,  0.01871072, -0.00329062, -0.01858585],
       [ 0.00349842, -0.00388213, -0.00597445,  0.00532572,  0.00590599],
       [-0.08447695,  0.03435381,  0.03832418, -0.02966298, -0.06234913],
       [ 0.00484249, -0.00667755,  0.00764245, -0.00129963, -0.00294504],
       [-0.00414873, -0.06328248,  0.07092509,  0.08421068, -0.0638046 ]])

In [0]:
s = np.diag(s)


In [0]:
V[:5, :5] 

array([[-0.02253238,  0.00349842, -0.08447695,  0.00484249, -0.00414873],
       [ 0.0134218 , -0.00388213,  0.03435381, -0.00667755, -0.06328248],
       [ 0.01871072, -0.00597445,  0.03832418,  0.00764245,  0.07092509],
       [-0.00329062,  0.00532572, -0.02966298, -0.00129963,  0.08421068],
       [-0.01858585,  0.00590599, -0.06234913, -0.00294504, -0.0638046 ]])

In [0]:
# Part (c)

In [0]:
W = np.matmul(U,np.sqrt(s))

In [0]:
W[:5, :5]

array([[-0.16691093,  0.09983719,  0.13945241, -0.02457296, -0.13973563],
       [ 0.02591488, -0.02887699, -0.04452798,  0.03977023,  0.04440354],
       [-0.62577173,  0.25553859,  0.28563296, -0.22151052, -0.46876499],
       [ 0.03587122, -0.04967053,  0.05695978, -0.00970509, -0.02214195],
       [-0.03073214, -0.4707226 ,  0.52860995,  0.62884958, -0.47970772]])

In [0]:
# serialize PMI Word embeddings to disk 

In [0]:
pickle.dump(W, gzip.open("pmi_learned.pkl.gz","wb"))

In [0]:
# Part (d)

In [0]:
# First, we need to decide how we will define "closeness". 
# We could try both cosine angular distance and euclidian distance and see
# choose the one that works better 

In [0]:
# Create a function that computes "closeness": cosine of the angle between two 
# normalized word vectors 

In [0]:
def compute_cosine(a, b):
    return (np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))

In [0]:
# Create a function that computes "closeness": euclidian distance between word 
# vectors 

In [0]:
def compute_sim(a, b):
    return scipy.spatial.distance.euclidean(a, b)

In [0]:
# Function that finds the 5 closest words in the embedding space 

In [0]:
def find_closest(W, V, w):
    dic = {}
    w_index = V.index(w)
    w_embed = W[w_index]
    for i in range(len(V)):
        if i != w_index:
            dic[i] = compute_cosine(w_embed, W[i])
    top_five = [top[0] for top in collections.Counter(dic).most_common(5)]
    return [V[ind] for ind in top_five]

In [0]:
# Let's first find 5 closest words for "physics"
print(find_closest(W, V, "physics"))

['mathematics', 'chemistry', 'mathematical', 'einstein', 'geometry']


In [0]:
print(find_closest(W, V, "republican"))

['candidate', 'presidential', 'bush', 'vice', 'democratic']


In [0]:
print(find_closest(W, V, "einstein"))

['physics', 'mathematics', 'albert', 'scientific', 'turing']


In [0]:
print(find_closest(W, V, "algebra"))

['algebraic', 'theorem', 'mathematical', 'geometry', 'element']


In [0]:
print(find_closest(W, V, "fish"))

['farming', 'rich', 'iron', 'wild', 'trees']


In [0]:
# The results show that even by common sense, the 5 closest context words 
# for the given words are mutually related. 

In [0]:
# Part (e)

In [0]:
# Create a function that computes operations on linear substructure 

In [0]:
def compute_sub(W, V, expression):  
    dic = {}
    substruc = 0
    for i in range(len(expression)):
        if i == 0:
            substruc -= W[V.index(expression[i])]
        else: 
            substruc += W[V.index(expression[i])]

    for k in range(len(V)):
        dic[k] = compute_cosine(substruc, W[k])
    top_five = [top[0] for top in collections.Counter(dic).most_common(5)]

    return [V[ind] for ind in top_five]

In [0]:
# Try analogy given in the exercise 

In [0]:
print(compute_sub(W, V, ["france", "paris", "england"]))

['england', 'london', 'bishop', 'canterbury', 'rome']


In [0]:
# Interestingly, we get "London" as one of the results. 

In [0]:
# Try 3 different analogies 

In [0]:
# try republican: democrat = conservative: ?

In [0]:
print(compute_sub(W, V, ["republican", "democrat", "conservative"]))

['anglicans', 'counted', 'theologians', 'abbots', 'merchants']


In [0]:
# Although we didn't get "liberal", we did get groups that often represent the
# liberal facade in religion, academia, and economy

In [0]:
# try president: politics = pope ?

In [0]:
print(compute_sub(W, V, ["president", "politics", "pope"]))

['byzantine', 'vi', 'vii', 'afonso', 'christianity']


In [0]:
# We get "christianity" which I believe is an amazing catch. Perhaps even better
# than the expected "religion"

In [0]:
# try  war: peace = fight: ?

In [0]:
print(compute_sub(W, V, ["war", "peace", "fight"]))

['reformed', 'apostolic', 'canonical', 'maintains', 'affiliated']


In [0]:
# We are testing very abstract relations here, but it seems that the algorithm 
# is captureing the differences in the abstract notions. The resulting words are 
# distantly related to peace, but they are clear opposites of fight. 